In [ ]:
# install & import the libraries needed
!pip3 install pandas
!pip3 install scikit-learn
from typing import List
from math import log
import numpy as np
import pandas as pd
import random
from sklearn.metrics.pairwise import cosine_similarity
# for printing out all the columns of a pandas dataframe https://towardsdatascience.com/how-to-show-all-columns-rows-of-a-pandas-dataframe-c49d4507fcf
pd.set_option('display.max_columns', None)

In [ ]:
# The corpus
CORPUS = [
    'this is the first document',
    'the first document is this',
    'this is the second document',
    'and this is the third document',
    'is this the first document'
]


In [ ]:
# we will reuse the functions we completed in the previous tutorial
def tf(term: str, doc: str) -> int:
        tf = sum([word == term for word in doc.split(" ")])  # word count
        return tf


def build_dtm(corpus: List[str]) -> pd.DataFrame:
    """
    we reuse what we implemented in previous tutorial
    :param corpus:
    :return:
    """
    # build vocabulary - use nested list comprehension, set, and list
    words = [
        word
        for doc in corpus
        for word in doc.split(" ")
    ]
    vocab = list(set(words))

    # populate dtm, ith a nested for loop
    dtm = []
    for doc in corpus:
        row = list()
        for term in vocab:
            term_freq = tf(term, doc)
            row.append(term_freq)
        dtm.append(row)

    # return dtm as a pandas dataframe
    dtm = pd.DataFrame(data=dtm, columns=vocab)
    return dtm



In [ ]:
# complete this function
def idf(term: str, corpus: List[str]) -> float:
    ### TODO 1 ###
    #idf = np.log(len(CORPUS)/(1+term))
    
    n = len(corpus)
    df = sum([
      term in doc.split(" ") #boolean 값이 나온다
      for doc corpus 
    ])

    idf  = log(n/(1+df))
    
    return idf

In [ ]:
def build_dtm_with_tfidf(corpus: List[str]) -> pd.DataFrame:
    # access the columns with df.columns
    # multiply idfs to each row with numpy's broadcast mul
    dtm = build_dtm(corpus) #tf
    term = list(dtm.columns)
    dtm_np = dtm.to_numpy()
    dtm_sum = np.sum(dtm_np,axis =0) 
    
    ### TODO 2 ####
    # reuse dtm to build dtm_tfidf
    # use dtm.columns, idf, np.array(), dtm.to_numpy() and broadcast multiplication
    idfs: List[float] = np.array([[idf(term,doc) for term in dtm_sum] for doc in corpus])
    '''
    idfs: List[float] = [idf(term,corpus) for term in dtm.columns]
    '''
    dtm_tfidf: List[List[float]] = dtm.to_numpy() * idfs

    '''
     dtm_tfidf: List[List[float]] = dtm.to_numpy() * idfs.to_numpy()
    '''
    
    
    
    #print(idfs)
    ############### 
    return pd.DataFrame(data=dtm_tfidf, columns=dtm.columns)


In [ ]:
# build dtm, and one with  
dtm = build_dtm(CORPUS)
dtm_tfidf = build_dtm_with_tfidf(CORPUS)
print(dtm)
display(dtm_tfidf)

   this  is  third  second  first  the  document  and
0     1   1      0       0      1    1         1    0
1     1   1      0       0      1    1         1    0
2     1   1      0       1      0    1         1    0
3     1   1      1       0      0    1         1    1
4     1   1      0       0      1    1         1    0


,this,is,third,second,first,the,document,and
0,-0.182322,-0.182322,0.000000,0.000000,0.223144,-0.182322,-0.182322,0.000000
1,-0.182322,-0.182322,0.000000,0.000000,0.223144,-0.182322,-0.182322,0.000000
2,-0.182322,-0.182322,0.000000,0.916291,0.000000,-0.182322,-0.182322,0.000000
3,-0.182322,-0.182322,0.916291,0.000000,0.000000,-0.182322,-0.182322,0.916291
4,-0.182322,-0.182322,0.000000,0.000000,0.223144,-0.182322,-0.182322,0.000000


다음과 같은 결과가 나와야 합니다 (단어의 순서는 달라도 괜찮습니다):
```
   this  is  third  second  first  the  document  and
0     1   1      0       0      1    1         1    0
1     1   1      0       0      1    1         1    0
2     1   1      0       1      0    1         1    0
3     1   1      1       0      0    1         1    1
4     1   1      0       0      1    1         1    0

      this	     is	    third	   second	   first	  the	document	and     
0	-0.182322	-0.182322	0.000000	0.000000	0.223144	-0.182322	-0.182322	0.000000
1	-0.182322	-0.182322	0.000000	0.000000	0.223144	-0.182322	-0.182322	0.000000
2	-0.182322	-0.182322	0.000000	0.916291	0.000000	-0.182322	-0.182322	0.000000
3	-0.182322	-0.182322	0.916291	0.000000	0.000000	-0.182322	-0.182322	0.916291
4	-0.182322	-0.182322	0.000000	0.000000	0.223144	-0.182322	-0.182322	0.000000

```

In [ ]:
# compare the two
print(cosine_similarity(dtm.to_numpy(), dtm.to_numpy()))
print(cosine_similarity(dtm_tfidf.to_numpy(), dtm_tfidf.to_numpy()))

[[1.         1.         0.8        0.73029674 1.        ]
 [1.         1.         0.8        0.73029674 1.        ]
 [0.8        0.8        1.         0.73029674 0.8       ]
 [0.73029674 0.73029674 0.73029674 1.         0.73029674]
 [1.         1.         0.8        0.73029674 1.        ]]
[[1.         1.         0.31538537 0.23104796 1.        ]
 [1.         1.         0.31538537 0.23104796 1.        ]
 [0.31538537 0.31538537 1.         0.10015744 0.31538537]
 [0.23104796 0.23104796 0.10015744 1.         0.23104796]
 [1.         1.         0.31538537 0.23104796 1.        ]]


다음과 같은 결과가 나와야 합니다:
```
[[1.         1.         0.8        0.73029674 1.        ]
 [1.         1.         0.8        0.73029674 1.        ]
 [0.8        0.8        1.         0.73029674 0.8       ]
 [0.73029674 0.73029674 0.73029674 1.         0.73029674]
 [1.         1.         0.8        0.73029674 1.        ]]

[[1.         1.         0.31538537 0.23104796 1.        ]
 [1.         1.         0.31538537 0.23104796 1.        ]
 [0.31538537 0.31538537 1.         0.10015744 0.31538537]
 [0.23104796 0.23104796 0.10015744 1.         0.23104796]
 [1.         1.         0.31538537 0.23104796 1.        ]]
```

## 다음의 질문에 답해주세요!

> `dtm`으로 구한 유사도 행렬 대비, `dtm_tfidf`로 구한 유사도 행렬은 어떤 점이 개선되었나요? 그 이유는?

> `dtm_tfidf`로도 해결할수 없는 문제를 발견할 수 있나요?

In [ ]:
#유사도를 측정하면서 다른 문장을 구분 할수 있다 

#단어의 순서를 구분할수 없다 